In [30]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

CONTENT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'vi_core_news_lg')
STATUS = data.LabelField(dtype = torch.float)

In [31]:
fields = [('content', CONTENT), ('status', STATUS)]

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = './input',
                                        train = 'train.csv',
                                        validation = 'validation.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [32]:
print(vars(train_data.examples[1]))

{'content': ['Happy', 'new', 'year', '.', '.', '!'], 'status': '1'}


In [33]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validating examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 904254
Number of validating examples: 65269
Number of testing examples: 50309


In [34]:
print(vars(train_data.examples[0]))

{'content': ['có', 'mẹ', 'nuôi'], 'status': '1'}


In [35]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [67]:
MAX_VOCAB_SIZE = 25_000

CONTENT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
STATUS.build_vocab(train_data)

print(f"Unique tokens in CONTENT vocabulary: {len(CONTENT.vocab)}")
print(f"Unique tokens in STATUS vocabulary: {len(STATUS.vocab)}")

Unique tokens in CONTENT vocabulary: 25002
Unique tokens in STATUS vocabulary: 2


In [37]:
print(CONTENT.vocab.freqs.most_common(20))

[('hay', 172285), ('quá', 126871), ('❤', 100177), ('.', 88164), ('️', 80642), ('nghe', 62144), ('anh', 57589), ('😍', 55500), ('bài', 50107), ('hát', 49343), (',', 48840), ('này', 42866), ('ơi', 42048), ('!', 41199), ('🥰', 40618), ('là', 38389), ('em', 36005), ('đi', 35059), (')', 35045), ('...', 33330)]


In [38]:
print(CONTENT.vocab.itos[:10])

['<unk>', '<pad>', 'hay', 'quá', '❤', '.', '️', 'nghe', 'anh', '😍']


In [39]:
print(STATUS.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [40]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.content),
    sort_within_batch = False,
    device = device)

In [41]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [42]:
INPUT_DIM = len(CONTENT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [43]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [44]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [45]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [46]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [47]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.content).squeeze(1)
        
        loss = criterion(predictions, batch.status)
        
        acc = binary_accuracy(predictions, batch.status)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.content).squeeze(1)
            
            loss = criterion(predictions, batch.status)
            
            acc = binary_accuracy(predictions, batch.status)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [49]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [50]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './model_rnn/model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 13m 24s
	Train Loss: 0.426 | Train Acc: 84.56%
	 Val. Loss: 0.650 |  Val. Acc: 67.25%
Epoch: 02 | Epoch Time: 14m 14s
	Train Loss: 0.425 | Train Acc: 84.73%
	 Val. Loss: 0.626 |  Val. Acc: 72.96%
Epoch: 03 | Epoch Time: 12m 52s
	Train Loss: 0.424 | Train Acc: 84.77%
	 Val. Loss: 0.608 |  Val. Acc: 76.44%
Epoch: 04 | Epoch Time: 12m 41s
	Train Loss: 0.424 | Train Acc: 84.80%
	 Val. Loss: 0.592 |  Val. Acc: 79.24%
Epoch: 05 | Epoch Time: 10m 22s
	Train Loss: 0.424 | Train Acc: 84.83%
	 Val. Loss: 0.578 |  Val. Acc: 80.51%
Epoch: 06 | Epoch Time: 9m 37s
	Train Loss: 0.423 | Train Acc: 84.85%
	 Val. Loss: 0.563 |  Val. Acc: 81.58%
Epoch: 07 | Epoch Time: 7m 52s
	Train Loss: 0.422 | Train Acc: 84.88%
	 Val. Loss: 0.540 |  Val. Acc: 82.39%
Epoch: 08 | Epoch Time: 11m 2s
	Train Loss: 0.420 | Train Acc: 84.67%
	 Val. Loss: 0.514 |  Val. Acc: 85.22%
Epoch: 09 | Epoch Time: 13m 59s
	Train Loss: 0.423 | Train Acc: 84.87%
	 Val. Loss: 0.507 |  Val. Acc: 85.00%
Epoch: 10 | E

In [68]:
model.load_state_dict(torch.load('./model_rnn/model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.525 | Test Acc: 82.67%
